In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import codecs
import csv
import datetime

In [2]:
# Kodeeringu parandamine UTF-8ks

with open('varem.csv') as f:
    tekst = f.readlines()

with open('varemkirjutatud.csv', mode='w', encoding="utf-8") as f:
    wr = csv.writer(f, delimiter=';')
    for rida in tekst:
        osad = rida.strip().split(';')
        wr.writerow(osad)

with open('hiljem.csv') as f:
    tekst = f.readlines()

with open('hiljemkirjutatud.csv', mode='w', encoding="utf-8") as f:
    wr = csv.writer(f, delimiter=';')
    for rida in tekst:
        osad = rida.strip().split(';')
        wr.writerow(osad)

In [3]:
# Andmete ühedamine ühte dataframe'i

too_early = pd.read_csv("varemkirjutatud.csv", sep=";")
too_late = pd.read_csv('hiljemkirjutatud.csv', sep=";")
data = pd.concat([too_early, too_late])
routes = data.route_short_name.unique()
print(len(data))

392435


In [4]:
# Veergude "estimated_departure_time" ja "deviation" formaadi parandamine
# Mõnes väljas esines probleem, et väärtuses esines juhuslik koma

# Veergude "id", "route_long_name", "user_id", "stop_code" kustutamine

new_estimated_departure_time = []
new_deviation = []
for dp, ed, dev in zip(data["departure_time"],data["estimated_departure_time"], data['deviation']):
    if "," in ed:
        #print(dp)
        #print(ed)
        dp_o = dp.split(":")
        ed_o = ed.split(",")
        ed_o = ed_o[0].split(":")
        dev = dev.split('.')[0]
        dev = dev.split(',')[0]
        if len(dev.split(":")) < 3:
            dev = "00:"+dev
        new_deviation += [dev]
        lst = []
        if ed_o[0][0] != '0' and ed_o[0][0] != '5':
            lst += [dp_o[0]]
            lst +=ed_o
        elif ed_o[0][0] == '0':     
            if dp_o[1][0] == '5':
                if dp_o[0] != '23':
                    abi = int(dp_o[0])+1
                else:
                    abi = '00'
                lst += [str(abi)]
                lst +=ed_o
            else:
                lst += [dp_o[0]]
                lst +=ed_o
        elif ed_o[0][0] == '5':     
            if dp_o[1][0] == '0':
                if dp_o[0] != '00':
                    abi = int(dp_o[0])-1
                else:
                    abi = '23'
                lst += [str(abi)]
                lst +=ed_o
            else:
                lst += [dp_o[0]]
                lst +=ed_o
        aeg = ":".join(lst)
        new_estimated_departure_time += [aeg]
    else:
        new_estimated_departure_time += [ed]
        dev = dev.split(".")[0]
        if len(dev.split(":")) < 3:
            dev = "00:"+dev
        new_deviation += [dev]
data['estimated_departure_time'] = new_estimated_departure_time
data['deviation'] = new_deviation
data.drop(['id', 'route_long_name', "user_id", "stop_code"], axis=1, inplace=True)
data.head()

,route_short_name,a_date,trip_departure_time,trip_id,stop_sequence,stop_id,stop_name,departure_time,estimated_departure_time,deviation,name,location_id
0,21,01.09.2019,04:20:00,2340749,1,2044001,Soola I,04:20:00,04:19:30,-00:00:30,Juht1,474TNS
1,21,01.09.2019,04:20:00,2340749,2,2043903,Aida,04:21:00,04:20:15,-00:00:45,Juht1,474TNS
2,21,01.09.2019,04:20:00,2340749,3,2044076,Silla II,04:22:00,04:21:48,-00:00:12,Juht1,474TNS
3,21,01.09.2019,04:20:00,2340749,4,2044071,Vaba,04:23:00,04:22:40,-00:00:20,Juht1,474TNS
4,21,01.09.2019,04:20:00,2340749,5,2044073,Tähe Erakliinik,04:25:00,04:24:09,-00:00:51,Juht1,474TNS


In [5]:
# Veeru "deviation" konverteerimine formaadist "hh:mm:ss" täisarvuks

new_deviation = []
for dev in data.deviation:
    as_int = int(dev[-2:]) + 60 * int(dev[-5:-3]) + 60 * 60 * int(dev[-8:-6])
    if dev[0] == "-":
        as_int = as_int * (-1)
    new_deviation.append(as_int)
data.deviation = new_deviation

In [6]:
# Ridade kustutamine, kus deviation suurem või väiksem 15 minutist

data = data[(data.deviation > 15*(-60)) & (data.deviation < 15*60)]

In [7]:
# Lisan veeru "wd", mis ütleb mitmes nädalapäev on
# Lisan veeru "date", mis on veerg "a_date" aga pandase datetime formaadis

data["date"] = pd.to_datetime(data["a_date"], format="%d.%m.%Y")
data["wd"] = data["date"].dt.dayofweek

In [8]:
# Trippide kustutamine, mille kohta esines andmeid ainult septembris.

# Bussigraafikud muutusid küll 1. septembril, ent hilinemisi mõõtvas süsteemis on ID-de muutumine
# toimunud hilinemisega, mistõttu on andmetes suur kogus halbu välju

bad_trips = []
for trip in data.trip_id.unique():
    dates = data[data.trip_id == trip].a_date.unique()
    dates = set([date[3:5] for date in dates])
    if dates == {'09'}:
        bad_trips.append(trip)
        
data = data[~data.trip_id.isin(bad_trips)]

In [9]:
# Andmetele veeru "total_stops" ja "route_direction_id" lisamine 

# Andmetes on kirjas ainult info liini kohta, kuid kuna liini erinevatel suundadel sõidavad
# erinevad tripid, on erinevad väljumisajad ja erinevad peatused, siis on oluline
# eristada ka liini sõitmise suunda.

# Veerg "total_stops" ütleb, mitu peatust on selle liinil kokku

data["route_direction_id"] = pd.Series()
data["total_stops"] = pd.Series()


# Suundade lisamise algoritmi põhimõte:
# 1) iga liini jaoks koostame võimalike suundade listi
# 2) iga selle liini tripi jaoks kontrollime iga võimaliku suuna jaoks suundade listis,
#    kas iga selle tripi n-s peatus on sama, mis n-s peatus sellel suunal
# 3) kui on, siis lisame selle tripi sinna suuna listi ning uuendame selle suuna kõiki puuduvate peatuste väärtuseid
#    nendega, mis esinevad lisatud tripil, kuid ei esinenud veel sellel suunal
# 4) kui ühegi suuna jaoks ei ole, siis loome uue suuna, mille peatuste listi loome selle tripi põhjal

for route_idx in routes:
    groups = []
    a = data[data["route_short_name"] == route_idx][["trip_id", "stop_sequence", "stop_name"]].groupby(["trip_id"]).agg(lambda x: list(x))
    for index, row in a.iterrows():
        no_group = True
        for group in groups:
            group_fit = True
            for i in range(len(row.stop_sequence)):
                if group[1][row.stop_sequence[i]] not in ["", row.stop_name[i]]:
                    group_fit = False
                    break
            if group_fit:
                group[0].add(index)
                for i in range(len(row.stop_sequence)):
                    group[1][row.stop_sequence[i]] = row.stop_name[i]
                no_group = False
                break

        if no_group:
            d = defaultdict(lambda: "")
            for i in range(len(row.stop_sequence)):
                d[row.stop_sequence[i]] = row.stop_name[i]
            groups.append([set(), d])
            groups[-1][0].add(index)    

    names = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k"]
    for i in range(len(groups)):  
        trips = groups[i][0]
        data.loc[data.trip_id.isin(trips), "total_stops"] = data[data.trip_id.isin(trips)].stop_sequence.max()
        data.loc[data.trip_id.isin(trips), "route_direction_id"] = route_idx + names[i]


In [10]:
# Veergude kustutamine, kus info on liini viimase peatuse kohta, kuna väljumine viimasest peatusest
# ei ole enam oluline

data = data[data.stop_sequence != data.total_stops]

In [11]:
# Kui sama trip hilines samal päeval samasse peatusesse korduvalt, siis on tegemist veaga mõõtmises ehk kustutame

data = data.drop_duplicates(['route_direction_id','stop_sequence', 'a_date', 'trip_id'],keep= 'last')

In [12]:
# Visuaalne kontroll

for c in data.columns:
    print(c)
    print(data[c].value_counts())

route_short_name
2     40203
6     31615
4     27966
10    23786
3     22151
11    21119
1     20085
8     16403
13    15501
12    14047
7     13782
9A     5123
9      3953
5      1891
25      861
22      762
21      490
69      155
E1       71
Name: route_short_name, dtype: int64
a_date
24.09.2019    9609
26.09.2019    9503
11.10.2019    9384
04.10.2019    9311
10.10.2019    9310
14.10.2019    8913
02.10.2019    8850
01.10.2019    8683
20.09.2019    8668
03.10.2019    8507
19.09.2019    8412
25.09.2019    8371
27.09.2019    8328
08.10.2019    8325
07.10.2019    8325
15.10.2019    8307
09.10.2019    8304
18.09.2019    8297
18.10.2019    8267
17.09.2019    8190
16.09.2019    8185
23.09.2019    8021
16.10.2019    7982
17.10.2019    7786
30.09.2019    7781
05.10.2019    5529
21.09.2019    5351
14.09.2019    5349
28.09.2019    4874
12.10.2019    4490
22.09.2019    4360
13.10.2019    4332
29.09.2019    4147
06.10.2019    4085
15.09.2019    3828
Name: a_date, dtype: int64
trip_departure_time

In [13]:
data.to_csv("puhastatud.csv")